# Named Entity Recognition on BC5CDR (Chemical + Disease Corpus) with BioBERT

---

[Github](https://github.com/eugenesiow/practical-ml/blob/master/notebooks/Named_Entity_Recognition_Mandarin_MSRA.ipynb) | More Notebooks @ [eugenesiow/practical-ml](https://github.com/eugenesiow/practical-ml)

---

Notebook to train/fine-tune a BioBERT model to perform named entity recognition (NER). 

The [dataset](https://github.com/shreyashub/BioFLAIR/tree/master/data/ner/bc5cdr) used is a pre-processed version of the BC5CDR (BioCreative V CDR task corpus: a resource for  relation extraction) dataset from [Li et al. (2016)](https://pubmed.ncbi.nlm.nih.gov/27161011/).

The current state-of-the-art model on this dataset is the NER+PA+RL model from [Nooralahzadeh et al. (2019)](https://www.aclweb.org/anthology/D19-6125/) has an F1-score of [**89.93%**](https://paperswithcode.com/sota/named-entity-recognition-ner-on-bc5cdr). The authors did not release the source code for the paper.

Our model trained on top of BioBERT has an F1-score of **89.3%** which is slightly worse than the state-of-the-art but almost as good as the #2 [BioFlair](https://github.com/shreyashub/BioFLAIR)!

The notebook is structured as follows:
* Setting up the GPU Environment
* Getting Data
* Training and Testing the Model
* Using the Model (Running Inference)

#### Task Description

> Named entity recognition (NER) is the task of tagging entities in text with their corresponding type. Approaches typically use BIO notation, which differentiates the beginning (B) and the inside (I) of entities. O is used for non-entity tokens.

# Setting up the GPU Environment

#### Ensure we have a GPU runtime

If you're running this notebook in Google Colab, select `Runtime` > `Change Runtime Type` from the menubar. Ensure that `GPU` is selected as the `Hardware accelerator`. This will allow us to use the GPU to train the model subsequently.

#### Install Dependencies and Restart Runtime

You might see the error `ERROR: google-colab X.X.X has requirement ipykernel~=X.X, but you'll have ipykernel X.X.X which is incompatible` after installing the dependencies. **This is normal** and caused by the `simpletransformers` library.

The **solution** to this will be to **reset the execution environment** now. Go to the menu `Runtime` > `Restart runtime` then continue on from the next section to download and process the data.

# Getting Data

#### Pulling the data from Github

The dataset, includes train, test and dev sets, which we pull from the [Github repository](https://github.com/shreyashub/BioFLAIR/tree/master/data/ner/bc5cdr).

In [4]:
import urllib.request
from pathlib import Path

def download_file(url, output_file):
  Path(output_file).parent.mkdir(parents=True, exist_ok=True)
  urllib.request.urlretrieve (url, output_file)

download_file('https://raw.githubusercontent.com/shreyashub/BioFLAIR/master/data/ner/bc5cdr/train.txt', 'train.txt')
download_file('https://raw.githubusercontent.com/shreyashub/BioFLAIR/master/data/ner/bc5cdr/test.txt', 'test.txt')
download_file('https://raw.githubusercontent.com/shreyashub/BioFLAIR/master/data/ner/bc5cdr/dev.txt', 'dev.txt')

Since the data is formatted in the CoNLL `BIO` type format (you can read more on the tagging format from this [wikipedia article](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging))), we need to format it into a `pandas` dataframe with the following function. The 3 important columns in the dataframe are a word token (for mandarin this is a single character), a `BIO` label and a sentence_id to differentiate samples/sentences.

In [5]:
import pandas as pd
def read_conll(filename):
    df = pd.read_csv(filename,
                    sep = '\t', header = None, keep_default_na = False,
                    names = ['words', 'pos', 'chunk', 'labels'],
                    quoting = 3, skip_blank_lines = False)
    df = df[~df['words'].astype(str).str.startswith('-DOCSTART- ')] # Remove the -DOCSTART- header
    df['sentence_id'] = (df.words == '').cumsum()
    return df[df.words != '']

Now we execute the function on the train, test and dev sets we have downloaded from Github. We also `.head()` the training set dataframe for the first 100 rows to check that the words, labels and sentence_id have been split properly.

In [6]:
train_df = read_conll('train.txt')
test_df = read_conll('test.txt')
dev_df = read_conll('dev.txt')
test_df.head(100)

,words,pos,chunk,labels,sentence_id
2,Famotidine,PROPN,O,I-Entity,1
3,-,PUNCT,O,O,1
4,associated,VERB,O,O,1
5,delirium,NOUN,O,I-Entity,1
6,.,PUNCT,O,O,1
...,...,...,...,...,...
101,elderly,ADJ,O,O,5
102,population,NOUN,O,O,5
103,seen,VERB,O,O,5
104,.,PUNCT,O,O,5


We now print out the statistics (number of sentences) of the train, dev and test sets.

In [10]:
data = [[train_df['sentence_id'].nunique(), test_df['sentence_id'].nunique(), dev_df['sentence_id'].nunique()]]

# Prints out the dataset sizes of train and test sets per label.
pd.DataFrame(data, columns=["Train", "Test", "Dev"])

,Train,Test,Dev
0,3942,4139,3949


# Training and Testing the Model

#### Set up the Training Arguments

We set up the training arguments. Here we train to 10 epochs to get accuracy close to the SOTA. The train, test and dev sets are relatively small so we don't have to wait too long. We set a sliding window as NER sequences can be quite long and because we have limited GPU memory we can't increase the `max_seq_length` too long.

In [24]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'train_batch_size': 32,
    'fp16': True,
    'output_dir': '/Users/kevinlu/Google Drive/MIDS/W266 Natural Langauge Processing/Project/SEQ/Output',
    'best_model_dir': '/Users/kevinlu/Google Drive/MIDS/W266 Natural Langauge Processing/Project/SEQ/Output/best_model/',
    'evaluate_during_training': True,
}

The following line of code saves (to the variable `custom_labels`) a set of all the NER tags/labels in the dataset.

In [25]:
custom_labels = list(train_df['labels'].unique())
print(custom_labels)

['I-Entity', 'O', 'B-Entity']


#### Train the Model

Once we have setup the `train_args` dictionary, the next step would be to train the model. We use the pre-trained BioBERT model (by [DMIS Lab, Korea University](https://huggingface.co/dmis-lab)) from the awesome [Hugging Face Transformers](https://github.com/huggingface/transformers) library as the base and use the [Simple Transformers library](https://simpletransformers.ai/docs/classification-models/) on top of it to make it so we can train the NER (sequence tagging) model with just a few lines of code.

In [26]:
from simpletransformers.seq2seq import Seq2SeqModel
from transformers import AutoTokenizer
import pandas as pd
import logging

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the bio BERT pre-trained model.
model = Seq2SeqModel(encoder_type="bert", encoder_name="dmis-lab/biobert-v1.1", decoder_name="dmis-lab/biobert-base-cased-v1.1", labels=custom_labels, args=train_args, use_cuda=False)

# Train the model
# https://simpletransformers.ai/docs/tips-and-tricks/#using-early-stopping
model.train_model(train_df, eval_data=dev_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, preds_list = model.eval_model(test_df)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /dmis-lab/biobert-v1.1/resolve/main/config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /dmis-lab/biobert-v1.1/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/dmis-lab/biobert-v1.1 HTTP/1.1" 200 624
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEB

The F1-score for the model is **89.3%** ('f1_score': 0.8927974947807933).

> For now thats the #3-ranked SOTA NER model on BC5CDR!

# Using the Model (Running Inference)

Running the model to do some predictions/inference is as simple as calling `model.predict(samples)`. First we get a sentence from the test set and print it out. Then we run the prediction on the sentence.

In [27]:
sample = test_df[test_df.sentence_id == 10].words.str.cat(sep=' ')
print(sample)

After a single oral dose of 4 mg / kg indomethacin ( IDM ) to sodium and volume depleted rats plasma renin activity ( PRA ) and systolic blood pressure fell significantly within four hours .


In [28]:
samples = [sample]
predictions, _ = model.predict(samples)
for idx, sample in enumerate(samples):
  print('{}: '.format(idx))
  for word in predictions[idx]:
    print('{}'.format(word))

INFO:simpletransformers.ner.ner_model: Converting to features started.
Running Prediction: 100%|██████████| 1/1 [00:00<00:00,  5.21it/s]

0: 
{'After': 'O'}
{'a': 'O'}
{'single': 'O'}
{'oral': 'O'}
{'dose': 'O'}
{'of': 'O'}
{'4': 'O'}
{'mg': 'O'}
{'/': 'O'}
{'kg': 'O'}
{'indomethacin': 'I-Entity'}
{'(': 'O'}
{'IDM': 'I-Entity'}
{')': 'O'}
{'to': 'O'}
{'sodium': 'I-Entity'}
{'and': 'O'}
{'volume': 'O'}
{'depleted': 'O'}
{'rats': 'O'}
{'plasma': 'O'}
{'renin': 'O'}
{'activity': 'O'}
{'(': 'O'}
{'PRA': 'O'}
{')': 'O'}
{'and': 'O'}
{'systolic': 'O'}
{'blood': 'O'}
{'pressure': 'O'}
{'fell': 'O'}
{'significantly': 'O'}
{'within': 'O'}
{'four': 'O'}
{'hours': 'O'}
{'.': 'O'}


We can connect to Google Drive with the following code to save any files you want to persist. You can also click the `Files` icon on the left panel and click `Mount Drive` to mount your Google Drive.

The root of your Google Drive will be mounted to `/content/drive/My Drive/`. If you have problems mounting the drive, you can check out this [tutorial](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166).

You can move the model checkpount files which are saved in the `/outputs/` directory to your Google Drive.

More Notebooks @ [eugenesiow/practical-ml](https://github.com/eugenesiow/practical-ml) and do drop us some feedback on how to improve the notebooks on the [Github repo](https://github.com/eugenesiow/practical-ml/).